In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import queue
import threading
import time
import json

In [ ]:
with open('yahoo_models.txt','r') as f:
    data=f.readlines()

    for line in data:
        infos=line.split('\n')[0].split('|')
        url=infos[0]
        typeDic=json.loads(infos[1])
        try:
            res = requests.get(url)
            res.encoding = 'utf-8'
            soup = BeautifulSoup(res.text, 'lxml') 
            for a in soup.select('.centercol a'):
                typeDic['type']=a.select_one('.title').text
                with open('yahoo_types_prac.txt','a') as f:
                    f.write('{}|{}\n'.format(a['href'],json.dumps(typeDic, ensure_ascii=False)))
        except Exception as e:
            print(e)

In [5]:
exitFlag = 0

class myThread (threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q
    def run(self):
        print ("Starting " + self.name)
        process_data(self.name, self.q)
        print ("Exiting " + self.name)

def process_data(threadName, q):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            data = q.get()
            queueLock.release()
            print ("%s processing %s" % (threadName, data))
        else:
            queueLock.release()
            time.sleep(1)

threadList = ["Thread-1", "Thread-2", "Thread-3"]
with open('yahoo_models.txt','r') as f:
    lines=f.readlines()
nameList = lines
queueLock = threading.Lock()
workQueue = queue.Queue(10)
threads = []
threadID = 1

# Create new threads
for tName in threadList:
    thread = myThread(threadID, tName, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1

# Fill the queue
queueLock.acquire()
for word in nameList:
    workQueue.put(word)
queueLock.release()

# Wait for queue to empty
while not workQueue.empty():
    pass

# Notify threads it's time to exit
exitFlag = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print ("Exiting Main Thread")

Starting Thread-1
Starting Thread-2
Starting Thread-3


KeyboardInterrupt: 